In [135]:
#pip install ee

In [136]:
#pip install geemap

In [137]:
import ee
import geemap
import pandas as pd
from matplotlib import pyplot
from io import StringIO


#ee.Authenticate()  # Only needed for the first time
ee.Initialize()
Map= geemap.Map()




In [168]:
lake = ee.FeatureCollection('projects/ee-touheda-khanom/assets/samplinglakes') \
    .filter(ee.Filter.eq('GNIS_Name', 'Big Moose Lake'))


Map.addLayer(lake)
Map.centerObject(lake, 16)
Map

Map(bottom=6112548.0, center=[43.83093869244275, -74.84826397158908], controls=(WidgetControl(options=['positi…

In [169]:
station = ee.Geometry.Point(-74.85423940003443,43.81880036802749)
Map.addLayer(station)
Map.centerObject(station, 16)


In [170]:
sentinel2_bands =['B1','B2','B3','B4','B5','B6','B8','B8A','B11']
STD_NAMES = ['Aerosols', 'Blue', 'Green', 'Red', 'RedEdge1','RedEdge2','RedEdge4','NIR','SWIR1']

# sentinel2_bands =['B1','B2','B3','B4']
# STD_NAMES = ['Aerosols', 'Blue', 'Green', 'Red']

In [172]:
def cdom(img):
    cdo = img.expression("(20.3 - 10. * (b2 / b3) - 2.4 * (b3 / b4))", {
        'b1': img.select('Aerosols'),
        'b2': img.select('Blue'),
        'b3': img.select('Green'),
        'b4': img.select('Red')
    }).rename("CO")

    bad2 = cdo.where((cdo.gte(0)), 1).rename("bad2")
    co = cdo.multiply(bad2).rename("CO")
    mask = co.neq(0)

    return img.updateMask(mask).clip(lake)


In [173]:
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10  # Bits 10 and 11 are clouds and cirrus, respectively.
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    maskedc = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(maskedc).divide(10000)
    names = image.bandNames()
#     return image.addBands(names,None, True).updateMask(maskedc).divide(10000)
#add all bands 

s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(lake) \
    .map(maskS2clouds) \
    .select(sentinel2_bands, STD_NAMES) \
    .map(cdom) 
     
     


s3 = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(lake) \
    .select(sentinel2_bands, STD_NAMES) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
    .map(cdom)

In [183]:
 first_image = s2.first()  # Get the first image from the collection
band_names = first_image.bandNames().getInfo()
print("Band Names:", band_names)

Band Names: ['Aerosols', 'Blue', 'Green', 'Red', 'RedEdge1', 'RedEdge2', 'RedEdge4', 'NIR', 'SWIR1']


In [175]:
#How many images?
print(s2.size().getInfo())


669


In [176]:
dates = s3.aggregate_array("system:time_start").getInfo()
#print(dates)
import datetime

dates = [datetime.datetime.fromtimestamp(int(date) / 1000).strftime('%Y-%m-%d') for date in dates]
print(dates)


['2019-03-27', '2019-04-16', '2019-06-15', '2019-07-10', '2019-07-10', '2019-07-25', '2019-08-04', '2019-08-14', '2019-08-14', '2019-09-13', '2019-09-18', '2019-09-18', '2019-10-08', '2019-10-08', '2019-10-13', '2019-10-13', '2019-10-28', '2020-03-01', '2020-03-01', '2020-03-16', '2020-04-25', '2020-05-05', '2020-05-05', '2020-05-20', '2020-05-20', '2020-06-04', '2020-06-09', '2020-06-09', '2020-06-19', '2020-06-19', '2020-07-04', '2020-07-04', '2020-07-09', '2020-07-09', '2020-07-19', '2020-07-19', '2020-07-24', '2020-07-24', '2020-08-23', '2020-08-23', '2020-09-12', '2020-09-12', '2020-09-22', '2020-09-22', '2020-10-17', '2020-10-17', '2020-11-06', '2020-11-21', '2021-02-04', '2021-03-21', '2021-03-21', '2021-04-05', '2021-04-05', '2021-04-10', '2021-04-10', '2021-06-19', '2021-06-24', '2021-06-24', '2021-07-19', '2021-08-03', '2021-10-12', '2021-11-06', '2021-11-06', '2021-11-21', '2022-01-20', '2022-02-09', '2022-03-06', '2022-03-16', '2022-03-21', '2022-04-30', '2022-04-30', '2022

In [177]:
import pandas as pd
import datetime

def station_mean(img):
    mean = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=lake, scale=30).get('CO')
    date = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd')
    return img.set('CDOM', mean).set('date', date)

station_reduced_imgs = s3.map(station_mean)
nested_list_date = station_reduced_imgs.aggregate_array('date')
nested_list_cdom = station_reduced_imgs.aggregate_array('CDOM')

date_list = nested_list_date.getInfo()
cdom_list = nested_list_cdom.getInfo()

# Convert dates to datetime objects
dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in date_list]

# Create DataFrame with 'date' and 'CDOM' columns
df = pd.DataFrame({'date': dates, 'CDOM': cdom_list})
df

EEException: Collection.reduceColumns: Error in map(ID=20190327T154901_20190327T155313_T18TVP):
Dictionary.get: Dictionary does not contain key: CO.

In [178]:
import pandas as pd
import datetime

def station_mean(img):
    mean = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=lake, scale=30).get('CO')
    date = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd')
    return img.set('CDOM', mean).set('date', date)

station_reduced_imgs = s2.map(station_mean)
nested_list_date = station_reduced_imgs.aggregate_array('date')
nested_list_cdom = station_reduced_imgs.aggregate_array('CDOM')

date_list = nested_list_date.getInfo()
cdom_list = nested_list_cdom.getInfo()

# Convert dates to datetime objects
dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in date_list]

# Create DataFrame with 'date' and 'CDOM' columns
df = pd.DataFrame({'date': dates, 'CDOM': cdom_list})
df

EEException: Collection.reduceColumns: Error in map(ID=20181217T155641_20181217T155849_T18TVP):
Date: Parameter 'value' is required.

In [ ]:
def station_mean(img):
    mean = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=lake, scale=30).get('CO')
    return img.set('CDOM', mean)

station_reduced_imgs = s2.map(station_mean)
nested_list = station_reduced_imgs.aggregate_array('CDOM')
df = pd.DataFrame(nested_list.getInfo(), columns=['CDOM'])


In [ ]:
pd.set_option('display.max_rows', None)

df

In [ ]:
df['date'] = pd.to_datetime(df['date'])

# Format 'date' column to exclude the time
df['date'] = df['date'].dt.date
df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
# Create a figure with subplots and set the dimensions to 15 x 7
fig, ax = plt.subplots(figsize=(15, 7))

# Create the plot by setting our dataframe to the data argument
sns.scatterplot(data=df, x='date', y='CDOM', ax=ax)

# Set the labels and title
ax.set_ylabel('CDOM', fontsize=20)
ax.set_xlabel('date', fontsize=20)
ax.set_title('Cdom Values over Time', fontsize=20)
ax.set_ylim(0,10)

# Format the x-axis tick labels to display only the year
# ax.xaxis.set_major_formatter(plt.FixedFormatter(df['date'].dt.year.unique()))

# Display the plot
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7), sharex=True)

# Seaborn scatter plot
sns.scatterplot(data=data, x=data.index, y='DOC', ax=ax1)
ax1.set_xlabel('DATE')
ax1.set_ylabel('DOC')
ax1.set_ylim(1,9)

ax1.set_title('DOC Values Over Time')

# Seaborn scatter plot
sns.scatterplot(data=df, x='date', y='CDOM', ax=ax2)
ax2.set_xlabel('DATE')
ax2.set_ylabel('CDOM')
ax2.set_ylim(0,10)
ax2.set_title('CDOM Values Over Time')

# Adjust the spacing between subplots
plt.tight_layout()

# Display the plot
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a figure and an Axes object
fig, ax = plt.subplots(figsize=(15, 7))

# Seaborn scatter plot for CDOM
sns.scatterplot(data=df, x='date', y='CDOM', ax=ax, label='CDOM', color='blue')

# Create a twin Axes object for DOC
ax2 = ax.twinx()

# Matplotlib scatter plot for DOC
ax2.scatter(data.index, data['DOC'], label='DOC', color='red')

# Set the labels and title
ax.set_xlabel('DATE')
ax.set_ylabel('CDOM', color='blue')
ax2.set_ylabel('DOC', color='red')
ax.set_title('CDOM vs DOC')

# Set different ranges for the y-axes
ax.set_ylim(4,10)  # Adjust the range for CDOM
ax2.set_ylim(1, 9)  # Adjust the range for DOC

# Add legends
ax.legend(loc='upper left')
ax2.legend(loc='upper right')

# Display the plot
plt.show()


In [ ]:
print(df['date'].dtype)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

df['date'] = pd.to_datetime(df['date'])

# Convert the date values to numeric format
df['date_numeric'] = pd.to_numeric(df['date']).astype(float)
data['index_numeric'] = pd.to_numeric(data.index).astype(float)

# Create a figure and an Axes object
fig, ax = plt.subplots(figsize=(15, 7))

# Matplotlib scatter plot for CDOM
ax.scatter(df['date'], df['CDOM'], label='CDOM', color='blue')

# Create a twin Axes object for DOC
ax2 = ax.twinx()

# Matplotlib scatter plot for DOC
ax2.scatter(data.index, data['DOC'], label='DOC', color='red')

# Calculate the linear regression line for CDOM
cdom_x = df['date_numeric']
cdom_y = df['CDOM']
cdom_slope, cdom_intercept = np.polyfit(cdom_x, cdom_y, 1)
cdom_line = cdom_slope * cdom_x + cdom_intercept
#ax.plot(df['date'], cdom_line, color='blue', label='CDOM Line Fit')

# Calculate the linear regression line for DOC
doc_x = data['index_numeric']
doc_y = data['DOC']
doc_slope, doc_intercept = np.polyfit(doc_x, doc_y, 1)
doc_line = doc_slope * doc_x + doc_intercept
ax2.plot(data.index, doc_line, color='red', label='DOC Line Fit')

# Set the labels and title
ax.set_xlabel('DATE')
ax.set_ylabel('CDOM', color='blue')
ax2.set_ylabel('DOC', color='red')
ax.set_title('CDOM vs DOC')

# Set different ranges for the y-axes
ax.set_ylim(4, 10)  # Adjust the range for CDOM
ax2.set_ylim(0, 10)  # Adjust the range for DOC

# Add legends
ax.legend(loc='upper left')
ax2.legend(loc='upper right')

# Display the plot
plt.show()


In [179]:
# Sort the doc DataFrame by date in ascending order
data = data.sort_index()

# Sort the keys in the 'date' column of df(CDOM) in ascending order
df.sort_values('date', inplace=True)

# Convert 'date' column in df(CDOM) to datetime dtype
df['date'] = pd.to_datetime(df['date'])



# Perform inner merge with seven-day window
merged_inner1 = pd.merge_asof(df, data, left_on='date', right_on='date', tolerance=pd.Timedelta(days=3))

#Get rid of the null values from doc
merged_inner1 = merged_inner1.dropna()

#print(merged_inner1)


NameError: name 'data' is not defined

In [180]:
# Drop duplicate rows based on DOC column, keeping the first occurrence
merged_inner1 = merged_inner1.drop_duplicates(subset=['DOC'])

# Print the updated DataFrame
#print(merged_inner1)


NameError: name 'merged_inner1' is not defined

In [ ]:
import matplotlib.pyplot as plt

# Plotting the scatter plot
plt.scatter(merged_inner1['DOC'], merged_inner1['CDOM'])

# Setting labels and title
plt.xlabel('DOC')
plt.ylabel('CDOM')
plt.title('Big Moose Lake Scatter Plot of DOC and CDOM for 3 days')
plt.ylim(4,10)
# Displaying the plot
plt.show()


In [ ]:
# Sort the doc DataFrame by date in ascending order
data = data.sort_index()

# Sort the keys in the 'date' column of df(CDOM) in ascending order
df.sort_values('date', inplace=True)

# Convert 'date' column in df(CDOM) to datetime dtype
df['date'] = pd.to_datetime(df['date'])



# Perform inner merge with seven-day window
merged_inner = pd.merge_asof(df, data, left_on='date', right_on='date', tolerance=pd.Timedelta(days=7))

#Get rid of the null values from doc
merged_inner = merged_inner.dropna()

#print(merged_inner)


In [ ]:
# Drop duplicate rows based on DOC column, keeping the first occurrence
merged_inner = merged_inner.drop_duplicates(subset=['DOC'])

# Print the updated DataFrame
#print(merged_inner)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.scatter(merged_inner['DOC'], merged_inner['CDOM'])
# Setting labels and title
plt.xlabel('DOC')
plt.ylabel('CDOM')
plt.title('BIG MOOSE LAKE Scatter Plot of DOC and CDOM for 7 days')
plt.ylim(4,10)
# Displaying the plot
plt.show()



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
# Create a figure with subplots and set the dimensions to 15 x 7
fig, ax = plt.subplots(figsize=(15, 7))

# Create the plot by setting our dataframe to the data argument
sns.scatterplot(data=merged_inner, x='date', y='CDOM', ax=ax)

# Set the labels and title
ax.set_ylabel('CDOM', fontsize=20)
ax.set_xlabel('date', fontsize=20)
ax.set_title('Merged Cdom Values over 7 days', fontsize=20)
ax.set_ylim(4,10)

# Format the x-axis tick labels to display only the year
# ax.xaxis.set_major_formatter(plt.FixedFormatter(df['date'].dt.year.unique()))

# Display the plot
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a figure and an Axes object
fig, ax = plt.subplots(figsize=(15, 7))

# Seaborn scatter plot for CDOM
sns.scatterplot(data=merged_inner, x='date', y='CDOM', ax=ax, label='CDOM', color='blue')

# Create a twin Axes object for DOC
ax2 = ax.twinx()

# Matplotlib scatter plot for DOC
ax2.scatter(merged_inner['date'], merged_inner['DOC'], label='DOC', color='red')

# Set the labels and title
ax.set_xlabel('DATE')
ax.set_ylabel('CDOM', color='blue')
ax2.set_ylabel('DOC', color='red')
ax.set_title('Merged CDOM vs DOC')

# Set different ranges for the y-axes
ax.set_ylim(4, 10)  # Adjust the range for CDOM
ax2.set_ylim(0, 10)  # Adjust the range for DOC

# Add legends
ax.legend(loc='upper left')
ax2.legend(loc='upper right')

# Display the plot
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Convert the date values to numeric format
merged_inner['date_numeric'] = pd.to_numeric(merged_inner['date']).astype(float)

# Create a figure and an Axes object
fig, ax = plt.subplots(figsize=(15, 7))

# Matplotlib scatter plot 
ax.scatter(merged_inner['date'], merged_inner['CDOM'], label='CDOM', color='blue')


# Create a twin Axes object for DOC
ax2 = ax.twinx()
ax2.scatter(merged_inner['date'], merged_inner['DOC'], label='DOC', color='red')


# Calculate the linear regression line for CDOM
cdom_x = merged_inner['date_numeric']
cdom_y = merged_inner['CDOM']
cdom_slope, cdom_intercept = np.polyfit(cdom_x, cdom_y, 1)
cdom_line = cdom_slope * cdom_x + cdom_intercept
#ax.plot(merged_inner['date'], cdom_line, color='blue', label='CDOM Line Fit')

# Calculate the linear regression line for DOC
doc_x = merged_inner['date_numeric']
doc_y = merged_inner['DOC']
doc_slope, doc_intercept = np.polyfit(doc_x, doc_y, 1)
doc_line = doc_slope * doc_x + doc_intercept
ax2.plot(merged_inner['date'], doc_line, color='red', label='DOC Line Fit')

# Set the labels and title
ax.set_xlabel('DATE')
ax.set_ylabel('CDOM', color='blue')
ax2.set_ylabel('DOC', color='red')
ax.set_title('Merged CDOM vs DOC for 7 days')

# Set different ranges for the y-axes
ax.set_ylim(4, 10)  # Adjust the range for CDOM
ax2.set_ylim(0, 10)  # Adjust the range for DOC

# Add legends
ax.legend(loc='upper left')
ax2.legend(loc='upper right')

# Display the plot
plt.show()
